# Numerical Integration

Over the summer, I worked through some course exercises in Python that were developed by Prof. John Kitchin at Carnegie Mellon.  

http://kitchingroup.cheme.cmu.edu/

I learned a lot from doing this, and I picked up some useful skills. One thing I noticed was that Prof. Kitchin makes heavy use of numerical quadrature in course, whereas I have barely used it in the past...though I can clearly see the need for it. I realized that I haven't taken full advantage of quadrature methods, but they are really nice tools to know for reactor design and analysis. The reason is that there are two situations that often arise in analysis of reactors that benefit from quadrature (numerical integration):

1. We frequently have to solve definite integrals for which the analytical solution is either impossible or sufficiently tedious, difficult, and error prone that it may as well be impossible. This is a good application for numerical integration using something like gaussian quadrature.
2. We often need to integrate discrete data points; this is typically done using quadrature methods like the trapezoidal rule or Simpson's rule.

These are built into most programming languages, and we should at least know how to use them when we need them. When you solve a definite integral on your calculator, they are most likely using some version of quadrature methods. We'll keep this exercise pretty brief.

Let's say we want to solve the following definite integral:

$$A = \int_0^{10} x^2 dx$$

Analytically, we know that the solution is:

$$\frac{x^3}{3} \bigg|_0^{10} = \frac{10^3}{3} - \frac{0^3}{3} = 333.33$$

We can also integrate this using gaussian quadrature, which provides a numerical approximation. We will not go into detail about how gaussian quadrature works, but if you're interested, it is basically done by calculating the function value at very specific locations of the independent variable, multiplying those values by very specific weight factors, and summing up all of the results:

https://en.wikipedia.org/wiki/Gaussian_quadrature

In Julia, easy gaussian quadrature is done using the QuadGK package:

    using Pkg
    Pkg.add("QuadGK")
    
Read more about it here:

https://github.com/JuliaMath/QuadGK.jl

The basic usage is that we first define the integrand as a function (the integrand is just what we are going to integrate). 

    function integrand(x)
        retval = x^2
    end

Then we can integrate directly with quadgk, which the basic syntax:

    integral, error = quadgk(function, lower limit, upper limit)
    
It returns both the value of the integral and the error associated with the numerical integration. For this example:

    int, err = quadgk(integrand, 0, 10)

**Simpsons and Trapezoidal Rules**

Simpson's rule and the trapezoidal rule work for discrete sets of data that we need to integrate.  So, for example, if I just calculated some discrete values of $y = x^2$ on the domain $x = [0,10]$:

    using Plots
    x = range(0, stop = 10, length = 15)
    y = x.^2    
    scatter(x,y)

So let's pretend we don't know that data was generated using the function $y = x^2$, and we just wanted to integrate it based on the discrete values of x and y that we have in our data set.  We would use either the trapezoidal rule or Simpson's rule.  Simpson's and Trapezoidal rules are both available in Julia through the NumericalIntegration.jl package.

    using Pkg
    Pkg.add("NumericalIntegration")

https://github.com/dextorious/NumericalIntegration.jl

If I wanted to calculate an approximate integral using the trapezoidal rule, would do so as follows:

    using NumericalIntegration
    integrate(x, y) #trapezoidal is default method
    
This will return the approximate integrated area of y(x)dx.

It is easy to change the method to Simpson's:

    integrate(x, y, SimpsonEven()) #note, data must be evenly spaced for Simpson's.

Finally, we commonly need to know the cumulative value of the integral at each value of the independent variable.  We can get this using the following syntax, which will only use a trazezoidal rule:

    ctrapz = cumul_integrate(x,y)
    plot(x, ctrapz, legend = nothing)
    
You'll see that this generates an array of integral values.  Basically, it keeps a running tab of the integrated area as you move through the set.  This can be very useful when generating things like distribution functions.